# LIMPIEZA TABLA COMPRAS Y CARGA A SILVER

In [0]:
from pyspark.sql.functions import col, regexp_replace, trim, when, current_timestamp, try_to_date

# 1. Cargar desde la tabla Bronze
df_bronze_compras = spark.read.table("workspace.bronze.compras")

# Función para limpiar decimales latinos con puntos de miles (1.400,00 -> 1400.00)
def fix_decimal_compras(column_name):
    return regexp_replace(
        regexp_replace(col(column_name), r'\.', ''), # Quita punto de miles
        r',', '.'                                    # Cambia coma por punto decimal
    ).cast("double")

# 2. PROCESAMIENTO SILVER
df_silver_compras = df_bronze_compras.select(
    # Paso 2 y 4: Fecha con formato dd/MM/yyyy y renombramiento
    try_to_date(trim(col("fecha_factura")), "dd/MM/yyyy").alias("fecha"),
    col("ruc").cast("string"),
    col("establecimiento").cast("string"),
    col("punto_emision").cast("string"),
    col("num_compra").alias("numero_compra"),
    col("autorizacion"),
    # Limpieza de categorías (Misma lógica que Productos)
    trim(regexp_replace(
        regexp_replace(
            regexp_replace(col("categoria"), "HIDRÁAULICA", "HIDRÁULICA"),
            "ILUMINACÍON", "ILUMINACIÓN"
        ), r'\s+$', ''
    )).alias("categoria"),
    col("codigo").alias("codigo_producto"),
    col("unidad_medida"),
    # Conversión de tipos numéricos (Paso 2)
    fix_decimal_compras("cantidad").cast("int").alias("cantidad"),
    fix_decimal_compras("costo").alias("costo_unitario"),
    fix_decimal_compras("precio").alias("precio_unitario"),
    fix_decimal_compras("subtotal").alias("subtotal"),
    fix_decimal_compras("iva").alias("iva"),
    fix_decimal_compras("total").alias("total")
).filter(
    # Paso 1: Filtrar filas donde la fecha sea válida
    (col("fecha").isNotNull())
).withColumn("fecha_carga_silver", current_timestamp())

# 3. Guardar en Silver
df_silver_compras.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.silver.compras")

print("✅ Tabla Compras procesada en Silver con éxito.")
#display(spark.read.table("workspace.silver.compras").limit(10))